# Demo 14: Convergence Analyzer

This notebook demonstrates the **Convergence Analyzer** for detecting when to stop iterative optimization.

## Why This Tool Exists

During optimization of `fast_zetas.py`, we:
- Applied 5 layers of corrections recursively
- **Manually** checked RMSE improvement after each layer
- **Manually** decided when improvement was "good enough"
- Had no systematic way to detect diminishing returns

This tool **automates that decision**.

## The Complete Optimization Toolkit

1. **Performance Profiler** → Identify bottlenecks
2. **Error Pattern Visualizer** → Discover corrections
3. **Formula Code Generator** → Generate production code
4. **Convergence Analyzer** → Decide when to stop ← **You are here**

## Key Insight

**Knowing when to stop is as important as knowing how to optimize.**

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from convergence_analyzer import (
    ConvergenceAnalyzer,
    ConvergenceVisualizer
)

print("✓ Imports successful")

## 1. Basic Convergence Analysis

Analyze a simple optimization history.

In [ ]:
# Simple RMSE history
rmse_history = [0.5, 0.3, 0.2, 0.15, 0.12, 0.10]

analyzer = ConvergenceAnalyzer(
    metric_history=rmse_history,
    metric_name="RMSE",
    lower_is_better=True
)

report = analyzer.analyze()
report.print_summary()

## 2. Real-World Example: Fast Zetas

Analyze the actual convergence from fast_zetas optimization.

In [ ]:
# Actual RMSE history from fast_zetas
fast_zetas_rmse = [0.512, 0.261, 0.145, 0.089, 0.056, 0.034]

analyzer = ConvergenceAnalyzer(
    metric_history=fast_zetas_rmse,
    metric_name="RMSE",
    lower_is_better=True,
    target_metric=0.01
)

report = analyzer.analyze()
report.print_summary()

print(f"\n✓ Total improvement: {report.total_improvement:.1%}")
print(f"✓ Convergence model: {report.convergence_rate.model_type}")
print(f"✓ Should stop: {report.stopping_recommendation.should_stop}")

## 3. Predict Future Improvements

Estimate what would happen if we continue optimizing.

In [ ]:
# Predict next 10 iterations
future_iters, future_rmse = analyzer.predict_future_improvements(n_future=10)

print(f"Current RMSE: {fast_zetas_rmse[-1]:.6f}")
print(f"\nPredicted future RMSE:")
for i, (iter_num, rmse) in enumerate(zip(future_iters[:5], future_rmse[:5]), 1):
    improvement = (fast_zetas_rmse[-1] - rmse) / fast_zetas_rmse[-1]
    print(f"  +{i} layers: {rmse:.6f} (improvement: {improvement:.1%})")

print(f"\nAfter 10 more layers: {future_rmse[-1]:.6f}")
total_improvement = (fast_zetas_rmse[-1] - future_rmse[-1]) / fast_zetas_rmse[-1]
print(f"Total additional improvement: {total_improvement:.1%}")

## 4. Detect Diminishing Returns

Find when improvements become negligible.

In [ ]:
# Detect with different thresholds
for threshold in [0.05, 0.02, 0.01]:
    dr = analyzer.detect_diminishing_returns(threshold=threshold)
    if dr:
        print(f"Threshold {threshold:.0%}: {dr.describe()}")
    else:
        print(f"Threshold {threshold:.0%}: No diminishing returns detected")

## 5. Cost-Benefit Analysis

Include iteration costs for economic analysis.

In [ ]:
# Iteration costs (e.g., computation time in seconds)
costs = [1.0, 1.2, 1.5, 2.0, 2.8, 4.0]

analyzer_with_costs = ConvergenceAnalyzer(
    metric_history=fast_zetas_rmse,
    metric_name="RMSE",
    lower_is_better=True,
    iteration_costs=costs
)

report = analyzer_with_costs.analyze()

print(f"Total cost: {report.total_cost:.2f} seconds")
print(f"\nCost-benefit ratios:")
if report.cost_benefit_ratios is not None:
    for i, ratio in enumerate(report.cost_benefit_ratios, 1):
        if np.isfinite(ratio):
            print(f"  Layer {i}: {ratio:.2f} seconds per unit improvement")

## 6. Visualize Convergence

Plot convergence curves with predictions.

In [ ]:
visualizer = ConvergenceVisualizer(analyzer)

fig, ax = plt.subplots(figsize=(12, 6))
visualizer.plot_convergence_curve(ax, show_prediction=True)
plt.tight_layout()
plt.show()

## 7. Improvement Rates

Visualize how improvement rate changes over iterations.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
visualizer.plot_improvement_rates(ax)
plt.tight_layout()
plt.show()

# Print improvement rates
improvements = analyzer.compute_improvement_rates()
print("\nImprovement rates:")
for i, rate in enumerate(improvements, 1):
    print(f"  Layer {i}: {rate:.2%}")

## 8. Compare Different Strategies

Compare convergence of multiple optimization approaches.

In [ ]:
# Simulate different strategies
strategy_a = [0.5, 0.25, 0.15, 0.10, 0.08, 0.07]  # Fast initial, then slow
strategy_b = [0.5, 0.4, 0.3, 0.2, 0.1, 0.05]      # Steady improvement
strategy_c = [0.5, 0.45, 0.42, 0.40, 0.39, 0.38]  # Slow convergence

analyzer_a = ConvergenceAnalyzer(strategy_a, "RMSE")
analyzer_b = ConvergenceAnalyzer(strategy_b, "RMSE")
analyzer_c = ConvergenceAnalyzer(strategy_c, "RMSE")

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 7))

for analyzer, name, color in [(analyzer_a, "Strategy A", 'blue'),
                               (analyzer_b, "Strategy B", 'green'),
                               (analyzer_c, "Strategy C", 'red')]:
    ax.plot(analyzer.iterations, analyzer.metric_history, 
            'o-', label=name, linewidth=2, markersize=6, color=color)

ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Convergence Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Compare final results
print("\nFinal comparison:")
for analyzer, name in [(analyzer_a, "Strategy A"),
                       (analyzer_b, "Strategy B"),
                       (analyzer_c, "Strategy C")]:
    report = analyzer.analyze()
    print(f"  {name}: {report.convergence_rate.model_type}, "
          f"improvement: {report.total_improvement:.1%}, "
          f"should stop: {report.stopping_recommendation.should_stop}")

## 9. Real-Time Monitoring

Simulate real-time convergence monitoring during optimization.

In [ ]:
# Simulate optimization loop
print("Simulating optimization with convergence monitoring:\n")

rmse_history = []
true_history = [0.5, 0.3, 0.2, 0.15, 0.13, 0.12, 0.115, 0.113, 0.112, 0.111]

for i, rmse in enumerate(true_history):
    rmse_history.append(rmse)
    print(f"Iteration {i}: RMSE = {rmse:.6f}")
    
    # Check convergence after 3 iterations
    if len(rmse_history) >= 3:
        analyzer = ConvergenceAnalyzer(rmse_history, "RMSE")
        report = analyzer.analyze()
        
        print(f"  → Predicted improvement: {report.stopping_recommendation.predicted_improvement:.2%}")
        
        if report.stopping_recommendation.should_stop:
            print(f"\n✓ STOPPING: {report.stopping_recommendation.reason}")
            print(f"  Confidence: {report.stopping_recommendation.confidence:.0%}")
            break
    
    print()

print(f"\nFinal RMSE: {rmse_history[-1]:.6f}")
print(f"Total iterations: {len(rmse_history)}")

## 10. Integration with Error Pattern Analyzer

Use convergence analysis during recursive refinement.

In [ ]:
from error_pattern_visualizer import ErrorPatternAnalyzer

# Create synthetic error
x = np.linspace(0, 10, 100)
actual = np.sin(x) + 0.1*x + 0.05*x**2
predicted = np.sin(x)

# Recursive refinement with convergence monitoring
analyzer = ErrorPatternAnalyzer(actual, predicted, x)
rmse_history = [analyzer.rmse]

print("Recursive refinement with convergence monitoring:\n")
print(f"Initial RMSE: {analyzer.rmse:.6f}")

for layer in range(10):
    report = analyzer.analyze_all()
    if len(report.suggestions) == 0:
        print("\nNo more corrections available")
        break
    
    analyzer = analyzer.apply_correction(report.suggestions[0])
    rmse_history.append(analyzer.rmse)
    
    print(f"Layer {layer+1}: RMSE = {analyzer.rmse:.6f}")
    
    # Check convergence
    if len(rmse_history) >= 3:
        conv_analyzer = ConvergenceAnalyzer(rmse_history, "RMSE")
        conv_report = conv_analyzer.analyze()
        
        if conv_report.stopping_recommendation.should_stop:
            print(f"\n✓ Convergence analyzer recommends stopping:")
            print(f"  {conv_report.stopping_recommendation.reason}")
            break

print(f"\nFinal RMSE: {rmse_history[-1]:.6f}")
print(f"Total layers: {len(rmse_history)}")
print(f"Improvement: {(rmse_history[0] - rmse_history[-1]) / rmse_history[0]:.1%}")

## Summary

The Convergence Analyzer provides:

1. **Convergence Detection** - Exponential, power law, linear, logarithmic
2. **Diminishing Returns** - Automatic threshold detection
3. **Future Prediction** - Extrapolate future improvements
4. **Stopping Recommendations** - Confident, actionable advice
5. **Cost-Benefit Analysis** - Economic optimization
6. **Visualization** - Clear convergence plots
7. **Real-Time Monitoring** - Use during optimization loops
8. **Integration** - Works with ErrorPatternAnalyzer

### When to Use

- During recursive formula refinement
- In iterative algorithm optimization
- When applying multiple correction layers
- To prevent over-optimization
- To save computation time

### The Complete Toolkit

```python
# 1. Profile performance
profiler = PerformanceProfiler()
bottlenecks = profiler.identify_bottlenecks()

# 2. Discover corrections
analyzer = ErrorPatternAnalyzer(actual, predicted, x)
report = analyzer.analyze_all()

# 3. Generate production code
generator = FormulaCodeGenerator(base_formula, name)
for correction in report.suggestions:
    generator.add_correction(correction)
generator.export_to_file("improved_formula.py")

# 4. Monitor convergence
conv_analyzer = ConvergenceAnalyzer(rmse_history, "RMSE")
if conv_analyzer.analyze().stopping_recommendation.should_stop:
    print("Optimization complete!")
```

**Key Insight**: Knowing when to stop is as important as knowing how to optimize!